# Merging some good baselines

In [21]:
import numpy as np
import pandas as pd 
from scipy.stats import linregress
from tqdm import tqdm
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
import os
from itertools import combinations
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss, log_loss, mean_absolute_error
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from xgboost import XGBRegressor
import glob
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor


from sklearn import *
#import redisAI
import glob
import optuna
from sklearn import ensemble
from sklearn.metrics import *
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
from torch import nn
import torch.optim as optim

from sklearn.neural_network import MLPRegressor  # 假设使用MLP回归器作为模型


# EDA for all

In [2]:
regular_m = pd.read_csv('./kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv')
tourney_m = pd.read_csv('./kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv')
teams_m = pd.read_csv('./kaggle/input/march-machine-learning-mania-2025/MTeams.csv')
# Load and Process Data Women's Tourney
regular_w = pd.read_csv('./kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv')
tourney_w = pd.read_csv('./kaggle/input/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv')
teams_w = pd.read_csv('./kaggle/input/march-machine-learning-mania-2025/WTeams.csv')
#print(teams_w.columns, regular_w.columns, tourney_m.columns)
# print(len(regular_m), len(tourney_m))


## EDA for XGBoost and the MLP method

In [3]:
# Getting all files
path = "./kaggle/input/march-machine-learning-mania-2025/**"
data = {p.split('/')[-1].split('.')[0].split('\\')[1] : pd.read_csv(p, encoding='latin-1') for p in glob.glob(path)}
df = data["SampleSubmissionStage2"]
# Creating year, left team, and right team columns
"""
df['Year'] = [int(yr[0:4]) for yr in df['ID']]
df['LTeam'] = [int(L[5:9]) for L in df['ID']]
df['RTeam'] = [int(R[10:14]) for R in df['ID']]
"""
df['RTeam'] = [int(R[10:14]) for R in df['ID']]
df['LTeam'] = [int(L[5:9]) for L in df['ID']]

df['LTeam']
df['ID'] # 从示例提交文件格式中获取左侧和右侧队伍的id
# Lots of feature selecting and engineering
teams = pd.concat([data['MTeams'], data['WTeams']])
teams_spelling = pd.concat([data['MTeamSpellings'], data['WTeamSpellings']])
teams_spelling = teams_spelling.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams_spelling.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams_spelling, how='left', on=['TeamID'])
del teams_spelling
season_cresults = pd.concat([data['MRegularSeasonCompactResults'], data['WRegularSeasonCompactResults']])
season_dresults = pd.concat([data['MRegularSeasonDetailedResults'], data['WRegularSeasonDetailedResults']])
tourney_cresults = pd.concat([data['MNCAATourneyCompactResults'], data['WNCAATourneyCompactResults']])
tourney_dresults = pd.concat([data['MNCAATourneyDetailedResults'], data['WNCAATourneyDetailedResults']])
slots = pd.concat([data['MNCAATourneySlots'], data['WNCAATourneySlots']])
seeds = pd.concat([data['MNCAATourneySeeds'], data['WNCAATourneySeeds']])
gcities = pd.concat([data['MGameCities'], data['WGameCities']])
seasons = pd.concat([data['MSeasons'], data['WSeasons']])

seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds[['Season', 'Seed', 'TeamID']].values}
cities = data['Cities']
sub = data['SampleSubmissionStage2']
del data

season_cresults['ST'] = 'S'
season_dresults['ST'] = 'S'
tourney_cresults['ST'] = 'T'
tourney_dresults['ST'] = 'T'
games = pd.concat((season_dresults, tourney_dresults), axis=0, ignore_index=True)# 只有2003年开始才有detailed results，这里舍弃了compact results
games.reset_index(drop=True, inplace=True)
games['WLoc'] = games['WLoc'].map({'A': 1, 'H': 2, 'N': 3})

games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)# 比赛id：年，1队，2队
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)# 1队，2队
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)# 1队
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)# 年 1队
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)

games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)

games['ScoreDiff'] = games['WScore'] - games['LScore']
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1) # 1队赢了没
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed']
games = games.fillna(-1)

c_score_col = ['NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl',
 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl',
 'LBlk', 'LPF'] # 选择和比赛得分相关的列
c_score_agg = ['sum', 'mean', 'median', 'max', 'min', 'std', 'skew', 'nunique']
gb = games.groupby(by=['IDTeams']).agg({k: c_score_agg for k in c_score_col}).reset_index()
# groupby 分组，同一支队的比赛会被分到一起
# agg 对于 c_score_col 中的每一列，分别计算每个分组（即每一对队伍）的这些聚合统计值。
# 聚合操作后，结果是一个 MultiIndex DataFrame。调用 reset_index() 是为了将其转换为普通的 DataFrame，方便后续处理。
gb.columns = [''.join(c) + '_c_score' for c in gb.columns]

games = games[games['ST']=='T']

sub['WLoc'] = 3
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] # 提取各种信息和添加种子特征
sub = sub.fillna(-1)

games = pd.merge(games, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')
sub = pd.merge(sub, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')
# 将比赛数据（games）与之前生成的统计特征数据（gb）通过IDTeams进行左连接。
# 将提交数据（sub）与统计特征数据（gb）通过IDTeams进行左连接。

col = [c for c in games.columns if c not in ['ID', 'DayNum', 'ST', 'Team1', 'Team2', 'IDTeams', 'IDTeam1', 'IDTeam2',
                                             'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'Pred', 'ScoreDiff', 'ScoreDiffNorm',
                                             'WLoc'] + c_score_col]


# XGB

## Params of XGB

In [36]:
# XGB parameters
param_grid = {
    'n_estimators': 5000,
    'learning_rate': 0.03,
    'max_depth': 6
}

## Predictions of XGBoost

In [ ]:
X = games[col].fillna(-1)
sub_X = sub[col].fillna(-1)

# Pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor(**param_grid, device="gpu", random_state=42))
])

# n_splits = 5  # 5折交叉验证
# tscv = TimeSeriesSplit(n_splits=n_splits)
# for fold, (train_index, test_index) in enumerate(tscv.split(X)):
#     print(f"Fold {fold + 1}/{n_splits}")
    
#     # 按索引拆分训练集和测试集
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = games['Pred'].iloc[train_index], games['Pred'].iloc[test_index]
#     #print(len(X_train), len(X_test), len(y_train), len(y_test))


# Fitting pipeline
#pipeline.fit(X, games['Pred'])

# Predicting games and submissions
#pred = pipeline.predict(X).clip(0.001, 0.999)
#sub_pred = pipeline.predict(sub_X).clip(0.001, 0.999)

# Cross validation (for the MSE)
#cv_scores = cross_val_score(pipeline, X, games['Pred'], cv=5, scoring="neg_mean_squared_error")
# 5 times of cross validation, 1. 1234, 5; 2: 1235,4, ...

#sub_pred = pipeline.predict(sub_X).clip(0.001, 0.999)

# submission_df = pd.DataFrame({
#     'ID': df['ID'],
#     'Pred': sub_pred
# })

# pred_of_XGBoost = submission_df

Fold 1/5
381 379 381 379
Fold 2/5
760 379 760 379
Fold 3/5
1139 379 1139 379
Fold 4/5
1518 379 1518 379
Fold 5/5
1897 379 1897 379


## Optuna adjustments for XGB

In [38]:
# Best Hyperparameters: {'n_estimators': 346, 'learning_rate': 0.010193615589609411, 'max_depth': 3}


def objective_xgb(trial):
    # 定义需要优化的参数
    param_grid = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),  # 迭代次数
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),  # 学习率
        'max_depth': trial.suggest_int('max_depth', 2, 7)  # 树的最大深度
    }

    # 定义 Pipeline
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('xgb', XGBRegressor(**param_grid, device="gpu", random_state=42))
    ])

    # 使用交叉验证计算均方误差
    cv_scores = cross_val_score(pipeline, X, games['Pred'], cv=5, scoring="neg_mean_squared_error")
    mse = -cv_scores.mean()  # 转换为 MSE
    return mse

# 创建 Optuna 研究对象
study_xgb = optuna.create_study(direction='minimize')  # 最小化 MSE

# 运行优化
study_xgb.optimize(objective_xgb, n_trials=50, show_progress_bar=True)  # 运行 50 次试验

# 获取最佳超参数
best_params_xgb = study_xgb.best_params
best_score_xgb = study_xgb.best_value

print("Best Hyperparameters:", best_params_xgb)
print("Best MSE:", best_score_xgb)



[I 2025-03-14 21:52:17,968] A new study created in memory with name: no-name-b444b53c-d1a0-4bc6-8260-c1ea4cce426b


  0%|          | 0/50 [00:00<?, ?it/s]

c:\Users\Li Nuo\.conda\envs\mmmcatboost\lib\site-packages\xgboost\core.py:158: UserWarning: [21:52:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[I 2025-03-14 21:52:29,263] Trial 0 finished with value: 0.21299650222658467 and parameters: {'n_estimators': 1492, 'learning_rate': 0.04203117420853208, 'max_depth': 3}. Best is trial 0 with value: 0.21299650222658467.
[I 2025-03-14 21:52:34,917] Trial 1 finished with value: 0.1964564134231959 and parameters: {'n_estimators': 727, 'learning_rate': 0.03786380291368044, 'max_depth': 3}. Best is trial 1 with value: 0.1964564134231959.
[I 2025-03-14 21:52:44,779] Trial 2 finished with value: 0.207970066826365 and parameters: {'n_estimators': 1326, 'learning_rate': 0.03998910005275811, 'max_depth': 3}. Best is trial 1 with value: 0.1964564134231959.
[I 2025-03-14 21:52:52,689] Trial 3 finished with value: 0.19841987526335386 and parameters: {'n_estimators': 314, 'learning_rate': 0.0567203728064781, 'max_depth': 7}. Best is trial 1 with value: 0.1964564134231959.
[I 2025-03-14 21:53:12,982] Trial 4 finished with value: 0.20963334788599336 and parameters: {'n_estimators': 1521, 'learning_rat

In [ ]:
# 运行优化后，获取所有试验
trials = study_xgb.trials

# 按照目标值（MSE）从小到大排序
sorted_trials = sorted(trials, key=lambda t: t.value)

# 选择前 5 次最佳试验
best_trials = sorted_trials[:5]

# 提取这 5 次试验的参数
best_params_list = [trial.params for trial in best_trials]

# 计算每个参数的平均值
average_params = {}
for param in best_params_list[0].keys():  # 假设所有试验的参数名称一致
    values = [trial[param] for trial in best_params_list]
    average_params[param] = np.mean(values)

print("Average of Best 5 Hyperparameters:", average_params)
# 运行优化后，获取所有试验
trials = study_xgb.trials

# 按照目标值（MSE）从小到大排序
sorted_trials = sorted(trials, key=lambda t: t.value)

# 选择前 5 次最佳试验
best_trials = sorted_trials[:5]

# 提取这 5 次试验的参数
best_params_list = [trial.params for trial in best_trials]

# 计算每个参数的平均值
average_params = {}
for param in best_params_list[0].keys():  # 假设所有试验的参数名称一致
    values = [trial[param] for trial in best_params_list]
    average_params[param] = np.mean(values)

print("Average of Best 5 Hyperparameters:", average_params)

Average of Best 5 Hyperparameters: {'n_estimators': 173.0, 'learning_rate': 0.028689023697776483, 'max_depth': 3.0}


In [46]:
st = study_xgb

optuna.visualization.plot_optimization_history(st).show()
optuna.visualization.plot_param_importances(st).show()
optuna.visualization.plot_contour(st).show()
# 绘制平行坐标图
optuna.visualization.plot_parallel_coordinate(st).show()

optuna.visualization.plot_slice(st).show()


## Optuna using timeseriessplit for XGB

### one-time predict


In [64]:
param_grid = {
    'n_estimators': 5000,
    'learning_rate': 0.03,
    'max_depth': 6
}

pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('xgb', XGBRegressor(**param_grid, device="gpu", random_state=42))
    ])

n_splits = 5  # 5折交叉验证
tscv = TimeSeriesSplit(n_splits=n_splits)
brier = []
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Fold {fold + 1}/{n_splits}")
    
    # 按索引拆分训练集和测试集
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = games['Pred'].iloc[train_index], games['Pred'].iloc[test_index]
    #print(len(X_train), len(X_test), len(y_train), len(y_test))
    pipeline.fit(X_train, y_train)
    pred = pipeline.predict(X_test).clip(0.001, 0.999)
    brier.append(brier_score_loss(y_test, pred))
brier

Fold 1/5
Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5


[0.24135187781680192,
 0.2344594494884052,
 0.21774566458487116,
 0.16814000974880242,
 0.18021951534721686]

In [69]:
sum(brier[-2:])/2

0.17417976254800965

In [71]:
# Best Hyperparameters: {'n_estimators': 346, 'learning_rate': 0.010193615589609411, 'max_depth': 3}


def objective_xgb(trial):
    # 定义需要优化的参数
    param_grid = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),  # 迭代次数
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),  # 学习率
        'max_depth': trial.suggest_int('max_depth', 2, 7)  # 树的最大深度
    }

    # 定义 Pipeline
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('xgb', XGBRegressor(**param_grid, device="gpu", random_state=42))
    ])

    n_splits = 5  # 5折交叉验证
    tscv = TimeSeriesSplit(n_splits=n_splits)
    brier = []
    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        #print(f"Fold {fold + 1}/{n_splits}")
        
        # 按索引拆分训练集和测试集
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = games['Pred'].iloc[train_index], games['Pred'].iloc[test_index]
        #print(len(X_train), len(X_test), len(y_train), len(y_test))
        pipeline.fit(X_train, y_train)
        pred = pipeline.predict(X_test).clip(0.001, 0.999)
        brier.append(brier_score_loss(y_test, pred))
    return sum(brier[-2:])/2


# 创建 Optuna 研究对象
study_xgb = optuna.create_study(direction='minimize')  # 最小化 MSE

# 运行优化
study_xgb.optimize(objective_xgb, n_trials=50, show_progress_bar=True)  # 运行 50 次试验

# 获取最佳超参数
best_params_xgb = study_xgb.best_params
best_score_xgb = study_xgb.best_value

print("Best Hyperparameters:", best_params_xgb)
print("Best MSE:", best_score_xgb)



[I 2025-03-15 00:22:52,518] A new study created in memory with name: no-name-65e068a8-5535-4f0b-9f6e-2ee4149ef4e8


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-03-15 00:22:55,656] Trial 0 finished with value: 0.166920474989452 and parameters: {'n_estimators': 186, 'learning_rate': 0.07696764681574009, 'max_depth': 5}. Best is trial 0 with value: 0.166920474989452.
[I 2025-03-15 00:23:13,119] Trial 1 finished with value: 0.17214869740506628 and parameters: {'n_estimators': 1133, 'learning_rate': 0.05593080305349766, 'max_depth': 6}. Best is trial 0 with value: 0.166920474989452.
[I 2025-03-15 00:23:15,194] Trial 2 finished with value: 0.160456515404241 and parameters: {'n_estimators': 323, 'learning_rate': 0.06100768701953475, 'max_depth': 2}. Best is trial 2 with value: 0.160456515404241.
[I 2025-03-15 00:23:32,038] Trial 3 finished with value: 0.17278602063033222 and parameters: {'n_estimators': 1128, 'learning_rate': 0.06270905619808692, 'max_depth': 6}. Best is trial 2 with value: 0.160456515404241.
[I 2025-03-15 00:23:50,254] Trial 4 finished with value: 0.169762435987886 and parameters: {'n_estimators': 1905, 'learning_rate': 0.0

In [ ]:
trials = study_xgb.trials

# 按照目标值（MSE）从小到大排序
sorted_trials = sorted(trials, key=lambda t: t.value)

# 选择前 5 次最佳试验
best_trials = sorted_trials[:5]

# 提取这 5 次试验的参数
best_params_list = [trial.params for trial in best_trials]
average_params = {}
for param in best_params_list[0].keys():  # 假设所有试验的参数名称一致
    values = [trial[param] for trial in best_params_list]
    average_params[param] = np.mean(values)

print("Average of Best 5 Hyperparameters:", average_params)
# Average of Best 5 Hyperparameters: {'n_estimators': 200.4, 'learning_rate': 0.032120027908225386, 'max_depth': 4.0}

Average of Best 5 Hyperparameters: {'n_estimators': 200.4, 'learning_rate': 0.032120027908225386, 'max_depth': 4.0}


# MLP

## Params of MLP

In [4]:
batch_size = 32  # Set batch size
lr = 0.01
n_layers = 2
num_epochs = 5  # Adjust the number of epochs based on performance
dropout_p = 0.3
hidden_layers_count = 100


## Funcs of MLP

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self, d_in, d_out, d_hidden, n_layers = n_layers, dropout_p = dropout_p):
        super().__init__()
        layers = [nn.Linear(d_in, d_hidden), nn.BatchNorm1d(d_hidden),
            nn.ReLU()]
        for layer in range(n_layers):
            layers += [nn.Linear(d_hidden, d_hidden), nn.BatchNorm1d(d_hidden),nn.ReLU(), nn.Dropout(p=dropout_p)]
        layers += [nn.Linear(d_hidden, d_out)]
        self.linear_relu_stack = nn.Sequential(*layers)

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
# Training function
def train(model, train_loader, loss_fn, optimizer, device):
    model.train()  # Set model to training mode
    total_loss = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)  # Move data to GPU if available

        optimizer.zero_grad()  # Reset gradients
        outputs = model(batch_X).squeeze()  # Forward pass (ensure output shape matches labels)
        loss = loss_fn(outputs, batch_y)  # Compute MSE loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        total_loss += loss.item()  # Accumulate loss
    
    return total_loss / len(train_loader)  # Return average loss per batch

# Evaluation function
def evaluate(model, val_loader, loss_fn, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    
    with torch.no_grad():  # Disable gradient computation
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X).squeeze()  # Ensure output shape matches labels
            
            loss = loss_fn(outputs, batch_y)
            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    return avg_loss  # Return MSE loss (lower is better)

## Predictions of the MLP method

In [7]:
X = games[col].fillna(-1)
y = games['Pred']

imputer = SimpleImputer(strategy='mean')  # 均值填充器
scaler = StandardScaler()# 标准化器
X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)


# turn numpy arrays into tensors
X_train_tensor = torch.tensor(X_scaled, dtype=torch.float32)  # Convert features to float
y_train_tensor = torch.tensor(y, dtype=torch.float32)  # Convert labels to long (for classification)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Train, valid split
train_size = int(0.8 * len(train_dataset))  # 80% for training
val_size = len(train_dataset) - train_size  # Remaining 20% for validation
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



model = NeuralNetwork(X_scaled.shape[1], 1, hidden_layers_count)
#print(model)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)  # Adjust learning rate as needed



# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, loss_fn, optimizer, device)
    val_loss = evaluate(model, val_loader, loss_fn, device)
    
    # print(f"Epoch {epoch+1}/{num_epochs}:")
    # print(f"  Train Loss (MSE): {train_loss:.4f}")
    # print(f"  Validation Loss (MSE): {val_loss:.4f}")

X_submit = sub[col].fillna(-1)
X_submit_imputed = imputer.transform(X_submit)
X_submit_scaled = scaler.transform(X_submit_imputed)


X_submit_tensor = torch.tensor(X_submit_scaled, dtype=torch.float32)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_submit_tensor = X_submit_tensor.to(device)

# Set model to evaluation mode
model.eval()

# Make predictions (disable gradients for efficiency)
with torch.no_grad():
    y_preds = model(X_submit_tensor).cpu().numpy()  # Move back to CPU for saving

sub['Pred'] = y_preds
# sub[['ID', 'Pred']].to_csv('submission.csv', index=False)

pred_of_MLP = sub[['ID', 'Pred']]


## correct train test split for mlp

In [ ]:



param_grid = {
    'alpha': 0.0001,
    'learning_rate_init': 0.001,
    'max_iter': 200,
    'hidden_layer_sizes': (1000, 1000)
}

# 假设 games 是你的原始数据 DataFrame，col 是特征列名列表
X = games[col].fillna(-1)  # 填充缺失值
y = games['Pred']  # 目标变量

# 定义预处理流程
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # 均值填充
    ('scaler', StandardScaler()),  # 标准化
    ('model', MLPRegressor(**param_grid))  # 神经网络回归器
])

# 设置时间序列交叉验证的折数
n_splits = 10
tscv = TimeSeriesSplit(n_splits=n_splits)

# 用于存储每折的 Brier 分数
brier_scores = []

# 进行时间序列交叉验证
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Fold {fold + 1}/{n_splits}")
    
    # 按索引拆分训练集和测试集
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 训练模型
    pipeline.fit(X_train, y_train)
    
    # 预测测试集
    pred = pipeline.predict(X_test).clip(0.001, 0.999)  # 将预测值限制在 [0.001, 0.999] 范围内
    
    # 计算 Brier 分数
    brier_score = brier_score_loss(y_test, pred)
    brier_scores.append(brier_score)
    
    print(f"Brier Score for Fold {fold + 1}: {brier_score:.4f}")

# 输出平均 Brier 分数
print(f"Average Brier Score: {np.mean(brier_scores):.4f}")

Fold 1/10
Brier Score for Fold 1: 0.2792
Fold 2/10
Brier Score for Fold 2: 0.2808
Fold 3/10
Brier Score for Fold 3: 0.2389
Fold 4/10
Brier Score for Fold 4: 0.2838
Fold 5/10
Brier Score for Fold 5: 0.3138
Fold 6/10
Brier Score for Fold 6: 0.2547
Fold 7/10
Brier Score for Fold 7: 0.2112
Fold 8/10
Brier Score for Fold 8: 0.2070
Fold 9/10
Brier Score for Fold 9: 0.2141
Fold 10/10
Brier Score for Fold 10: 0.2347
Average Brier Score: 0.2518


## Optuna for MLP

In [ ]:
X = games[col].fillna(-1)
y = games['Pred']

def objective_mlp(trial):
    # 定义需要优化的参数
    params = {
    'batch_size': trial.suggest_int('batch_size', 16, 64),  # 批量大小
    'lr': trial.suggest_float('lr', 0.001, 0.1),  # 学习率
    'n_layers': trial.suggest_int('n_layers', 2, 5),  # 网络层数
    'num_epochs': trial.suggest_int('num_epochs', 5, 30),  # 训练轮数
    'dropout_p': trial.suggest_float('dropout_p', 0.1, 0.5),  # Dropout 概率
    'hidden_layers_count': trial.suggest_int('hidden_layers_count', 50, 200)  # 隐藏层神经元数量
    }

    batch_size, lr, n_layers, num_epochs, dropout_p, hidden_layers_count = params.values()

    try:

        imputer = SimpleImputer(strategy='mean')  
        scaler = StandardScaler()
        X_imputed = imputer.fit_transform(X)
        X_scaled = scaler.fit_transform(X_imputed)


        # turn numpy arrays into tensors
        X_train_tensor = torch.tensor(X_scaled, dtype=torch.float32)  # Convert features to float
        y_train_tensor = torch.tensor(y, dtype=torch.float32)  # Convert labels to long (for classification)
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

        # Train, valid split
        train_size = int(0.8 * len(train_dataset))  # 80% for training
        val_size = len(train_dataset) - train_size  # Remaining 20% for validation
        train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



        model = NeuralNetwork(X_scaled.shape[1], 1, hidden_layers_count, n_layers=n_layers, dropout_p = dropout_p)
        #print(model)
        loss_fn = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)  # Adjust learning rate as needed



        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # Training loop

        for epoch in range(num_epochs):
            train_loss = train(model, train_loader, loss_fn, optimizer, device)
            val_loss = evaluate(model, val_loader, loss_fn, device)
            
            # print(f"Epoch {epoch+1}/{num_epochs}:")
            # print(f"  Train Loss (MSE): {train_loss:.4f}")
            # print(f"  Validation Loss (MSE): {val_loss:.4f}")
    except:
        return 1

    # X_submit = sub[col].fillna(-1)
    # X_submit_imputed = imputer.transform(X_submit)
    # X_submit_scaled = scaler.transform(X_submit_imputed)


    # X_submit_tensor = torch.tensor(X_submit_scaled, dtype=torch.float32)

    # # Move to GPU if available
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # X_submit_tensor = X_submit_tensor.to(device)

    # # Set model to evaluation mode
    # model.eval()

    # # Make predictions (disable gradients for efficiency)
    # with torch.no_grad():
    #     y_preds = model(X_submit_tensor).cpu().numpy()  # Move back to CPU for saving

    # sub['Pred'] = y_preds
    # # sub[['ID', 'Pred']].to_csv('submission.csv', index=False)

    # pred_of_MLP = sub[['ID', 'Pred']]
    
    return val_loss

# 创建 Optuna 研究对象
study_MLP = optuna.create_study(direction='minimize')  # 最小化 MSE

# 运行优化
study_MLP.optimize(objective_mlp, n_trials=50, show_progress_bar=True)  # 运行 50 次试验

# 获取最佳超参数
best_params_mlp = study_MLP.best_params
best_score_mlp = study_MLP.best_value

print("Best Hyperparameters:", best_params_mlp)
print("Best MSE:", best_score_mlp)


[I 2025-03-14 22:03:37,540] A new study created in memory with name: no-name-e10bf1ce-3d0b-4096-85b7-942fcaa89978


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-03-14 22:03:42,139] Trial 0 finished with value: 0.2615047406691771 and parameters: {'batch_size': 36, 'lr': 0.07578533871859953, 'n_layers': 5, 'num_epochs': 17, 'dropout_p': 0.38883073866558415, 'hidden_layers_count': 107}. Best is trial 0 with value: 0.2615047406691771.
[I 2025-03-14 22:03:48,989] Trial 1 finished with value: 0.2052613378598772 and parameters: {'batch_size': 16, 'lr': 0.03575495898489484, 'n_layers': 3, 'num_epochs': 16, 'dropout_p': 0.32257728146856257, 'hidden_layers_count': 63}. Best is trial 1 with value: 0.2052613378598772.
[I 2025-03-14 22:03:55,652] Trial 2 finished with value: 0.2546239058519232 and parameters: {'batch_size': 16, 'lr': 0.0565298130674371, 'n_layers': 2, 'num_epochs': 18, 'dropout_p': 0.4282187223698366, 'hidden_layers_count': 65}. Best is trial 1 with value: 0.2052613378598772.
[I 2025-03-14 22:03:58,186] Trial 3 finished with value: 0.2018164818485578 and parameters: {'batch_size': 41, 'lr': 0.040652045877930434, 'n_layers': 2, 'num

In [ ]:
# 运行优化后，获取所有试验
trials = study_MLP.trials

# 按照目标值（MSE）从小到大排序
sorted_trials = sorted(trials, key=lambda t: t.value)

# 选择前 5 次最佳试验
best_trials = sorted_trials[:5]

# 提取这 5 次试验的参数
best_params_list = [trial.params for trial in best_trials]

# 计算每个参数的平均值
average_params = {}
for param in best_params_list[0].keys():  # 假设所有试验的参数名称一致
    values = [trial[param] for trial in best_params_list]
    average_params[param] = np.mean(values)

print("Average of Best 5 Hyperparameters:", average_params)
# Average of Best 5 Hyperparameters: {'batch_size': 30.4, 'lr': 0.03774355556908798, 'n_layers': 2.0, 'num_epochs': 24.0, 'dropout_p': 0.36610595571302734, 'hidden_layers_count': 102.6}


Average of Best 5 Hyperparameters: {'batch_size': 30.4, 'lr': 0.03774355556908798, 'n_layers': 2.0, 'num_epochs': 24.0, 'dropout_p': 0.36610595571302734, 'hidden_layers_count': 102.6}


In [49]:
st = study_MLP

optuna.visualization.plot_optimization_history(st).show()
optuna.visualization.plot_param_importances(st).show()
optuna.visualization.plot_contour(st).show()
# 绘制平行坐标图
optuna.visualization.plot_parallel_coordinate(st).show()

optuna.visualization.plot_slice(st).show()

# Catboost

## EDA for catboost

In [7]:
# men_results = pd.concat([regular_m, tourney_m])[['Season', 'WTeamID', 'LTeamID']].copy()
men_results = tourney_m[['Season', 'WTeamID', 'LTeamID']].copy()
men_results['Result'] = 1 
m_inv = men_results.copy()
m_inv[['WTeamID', 'LTeamID']] = men_results[['LTeamID', 'WTeamID']].values
m_inv['Result'] = 0  # Loss label
men_results_final = pd.concat([men_results, m_inv], ignore_index=True)
# women_results = pd.concat([regular_w, tourney_w])[['Season', 'WTeamID', 'LTeamID']].copy()
women_results = tourney_w[['Season', 'WTeamID', 'LTeamID']].copy()
women_results['Result'] = 1 
w_inv = women_results.copy()
w_inv[['WTeamID', 'LTeamID']] = women_results[['LTeamID', 'WTeamID']].values
w_inv['Result'] = 0  # Loss label
women_results_final = pd.concat([women_results, w_inv], ignore_index=True)
# 居然只用季后赛作为训练集吗，常规赛也值得尝试
# 难道说模型的处理数据能力不足

all_results = pd.concat([men_results_final, women_results_final], ignore_index=True)


## Params of Catboost

In [23]:
catboost_params = {
    'iterations': 1000,
    'depth': 6,
    'learning_rate': 0.1,
    'loss_function':
    'Logloss',
    'verbose': 200
}

## Predictions of Catboost

In [24]:
# Train model
all_results = pd.concat([men_results_final, women_results_final], ignore_index=True)

X = all_results[['Season', 'WTeamID', 'LTeamID']]
y = all_results['Result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostClassifier(**catboost_params)
model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Evaluate model
preds = model.predict_proba(X_test)[:, 1]
print(f"Brier Score: {brier_score_loss(y_test, preds)}")

# Generate submission
input_folder = r"./kaggle/input/march-machine-learning-mania-2025"
required_ids_df = pd.read_csv(os.path.join(input_folder, "SampleSubmissionStage2.csv"))
team_ids = []
for game_id in required_ids_df['ID']:
    # 分割ID字符串，获取队伍ID
    team_ids.extend(game_id.split('_')[1:])  # 去掉年份和下划线

# 使用集合去重，然后转换为numpy数组并排序
all_teams = np.sort(np.unique(team_ids))
pairings = list(combinations(all_teams, 2))

# Load required matchup IDs
required_ids_df = pd.read_csv(os.path.join(input_folder, "SampleSubmissionStage2.csv"))
required_ids = set(required_ids_df['ID'])

def create_submission(pairings, season=2025, max_rows=131407):
    submission = []
    for (team1, team2) in pairings:
        matchup_id = f"{season}_{min(team1, team2)}_{max(team1, team2)}"
        if matchup_id in required_ids:
            input_data = pd.DataFrame({'Season': [season], 'WTeamID': [min(team1, team2)], 'LTeamID': [max(team1, team2)]})
            pred = model.predict_proba(input_data)[0, 1] if len(input_data) > 0 else 0.5
            submission.append([matchup_id, pred])
    submission_df = pd.DataFrame(submission, columns=["ID", "Pred"])
    print(f"Submission file has {submission_df.shape[0]} rows.")
    return submission_df

submission_df = create_submission(pairings)

pred_of_Catboost = submission_df

# 居然用更多的数据效果会更差吗

0:	learn: 0.6918510	test: 0.6925788	best: 0.6925788 (0)	total: 134ms	remaining: 2m 13s
200:	learn: 0.5864055	test: 0.6387680	best: 0.6385212 (199)	total: 601ms	remaining: 2.39s
400:	learn: 0.5197741	test: 0.6253272	best: 0.6253272 (400)	total: 1.05s	remaining: 1.57s
600:	learn: 0.4741209	test: 0.6298693	best: 0.6245255 (415)	total: 1.48s	remaining: 982ms
800:	learn: 0.4352536	test: 0.6313251	best: 0.6245255 (415)	total: 1.97s	remaining: 489ms
999:	learn: 0.4046832	test: 0.6348452	best: 0.6245255 (415)	total: 2.39s	remaining: 0us

bestTest = 0.6245255022
bestIteration = 415

Shrink model to first 416 iterations.
Brier Score: 0.21737738992316263
Submission file has 131407 rows.


## Optuna for Catboost

In [28]:
X = all_results[['Season', 'WTeamID', 'LTeamID']]
y = all_results['Result']

X = games[col].fillna(-1)
y = games['Pred']

def objective_catboost(trial):
    # 定义需要优化的参
    param_grid = {
        'iterations': trial.suggest_int('iterations', 100, 2000),  # 迭代次数
        'depth': trial.suggest_int('depth', 3, 10),  # 树的深度
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),  # 学习率
        'loss_function': trial.suggest_categorical('loss_function', ['Logloss']),  # 损失函数
    }

    
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('catboost', CatBoostClassifier(**param_grid, verbose=False, task_type='GPU'))
    ])


    n_splits = 5  # 5折交叉验证
    tscv = TimeSeriesSplit(n_splits=n_splits)
    brier = []
    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        #print(f"Fold {fold + 1}/{n_splits}")
        
        # 按索引拆分训练集和测试集
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        #print(len(X_train), len(X_test), len(y_train), len(y_test))
        pipeline.fit(X_train, y_train)
        pred = pipeline.predict(X_test).clip(0.001, 0.999)
        brier.append(brier_score_loss(y_test, pred))
    return sum(brier[-2:])/2

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # model = CatBoostClassifier(**catboost_params)
    # model.fit(X_train, y_train, eval_set=(X_test, y_test))

    # # Evaluate model
    # preds = model.predict_proba(X_test)[:, 1]
    # #print(f"Brier Score: {brier_score_loss(y_test, preds)}")


    # return brier_score_loss(y_test, preds)

# 创建 Optuna 研究对象
study_catboost = optuna.create_study(direction='minimize')  # 最小化 MSE

# 运行优化
study_catboost.optimize(objective_catboost, n_trials=50, show_progress_bar=True)  # 运行 50 次试验

# 获取最佳超参数
best_params_catboost = study_catboost.best_params
best_score_catboost = study_catboost.best_value

print("Best Hyperparameters:", best_params_catboost)
print("Best MSE:", best_score_catboost)

# def objective_xgb(trial):
#     # 定义需要优化的参数
#     param_grid = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 2000),  # 迭代次数
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),  # 学习率
#         'max_depth': trial.suggest_int('max_depth', 2, 7)  # 树的最大深度
#     }

#     # 定义 Pipeline
#     pipeline = Pipeline([
#         ('imputer', SimpleImputer(strategy='mean')),
#         ('scaler', StandardScaler()),
#         ('xgb', XGBRegressor(**param_grid, device="gpu", random_state=42))
#     ])

#     n_splits = 5  # 5折交叉验证
#     tscv = TimeSeriesSplit(n_splits=n_splits)
#     brier = []
#     for fold, (train_index, test_index) in enumerate(tscv.split(X)):
#         #print(f"Fold {fold + 1}/{n_splits}")
        
#         # 按索引拆分训练集和测试集
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = games['Pred'].iloc[train_index], games['Pred'].iloc[test_index]
#         #print(len(X_train), len(X_test), len(y_train), len(y_test))
#         pipeline.fit(X_train, y_train)
#         pred = pipeline.predict(X_test).clip(0.001, 0.999)
#         brier.append(brier_score_loss(y_test, pred))
#     return sum(brier[-2:])/2


# # 创建 Optuna 研究对象
# study_xgb = optuna.create_study(direction='minimize')  # 最小化 MSE

# # 运行优化
# study_xgb.optimize(objective_xgb, n_trials=50, show_progress_bar=True)  # 运行 50 次试验

# # 获取最佳超参数
# best_params_xgb = study_xgb.best_params
# best_score_xgb = study_xgb.best_value

# print("Best Hyperparameters:", best_params_xgb)
# print("Best MSE:", best_score_xgb)



[I 2025-03-16 14:29:51,055] A new study created in memory with name: no-name-65d43ea5-cae8-48e2-855e-9bc8c03856d9


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-03-16 14:32:19,953] Trial 0 finished with value: 0.2725418970976254 and parameters: {'iterations': 1069, 'depth': 6, 'learning_rate': 0.056169732228216085, 'loss_function': 'Logloss'}. Best is trial 0 with value: 0.2725418970976254.
[I 2025-03-16 14:34:26,647] Trial 1 finished with value: 0.24752606596306073 and parameters: {'iterations': 1118, 'depth': 3, 'learning_rate': 0.026939835555130392, 'loss_function': 'Logloss'}. Best is trial 1 with value: 0.24752606596306073.
[I 2025-03-16 14:44:52,733] Trial 2 finished with value: 0.2593756701846966 and parameters: {'iterations': 1742, 'depth': 8, 'learning_rate': 0.02377463150743506, 'loss_function': 'Logloss'}. Best is trial 1 with value: 0.24752606596306073.
[I 2025-03-16 14:58:46,427] Trial 3 finished with value: 0.2606922928759895 and parameters: {'iterations': 1271, 'depth': 9, 'learning_rate': 0.032263726584045076, 'loss_function': 'Logloss'}. Best is trial 1 with value: 0.24752606596306073.
[I 2025-03-16 15:10:54,587] Trial

In [33]:
trials = study_catboost.trials

# 按照目标值（MSE）从小到大排序
sorted_trials = sorted(trials, key=lambda t: t.value)

# 选择前 5 次最佳试验
best_trials = sorted_trials[:5]

# 提取这 5 次试验的参数
best_params_list = [trial.params for trial in best_trials]
average_params = {}
for param in best_params_list[0].keys():  # 假设所有试验的参数名称一致
    values = [trial[param] for trial in best_params_list]
    # import ipdb; ipdb.set_trace()
    if all(isinstance(value, (int, float)) for value in values):
        average_params[param] = np.mean(values)
    else:
        # 如果参数不是数值类型，可以选择跳过或单独处理
        average_params[param] = None  # 或者使用其他方式处理

print("Average of Best 5 Hyperparameters:", average_params)
# Average of Best 5 Hyperparameters: {'n_estimators': 200.4, 'learning_rate': 0.032120027908225386, 'max_depth': 4.0}

Average of Best 5 Hyperparameters: {'iterations': 322.8, 'depth': 3.4, 'learning_rate': 0.025096132434267882, 'loss_function': None}


# ELO

## Params of ELO

In [4]:
init_rating_m = 1200
init_rating_w = 1250
k_m = 125
k_w = 190
alpha_m = None
alpha_w = None
weights_regular_m = 1
weights_tournament_m = 0.7
weights_regular_w = 0.95
weights_tournament_w = 1

# Best Hyperparameters: {'init_rating_w': 1283, 'k_w': 129, 'weights_regular_w': 1.4277066442472455, 'weights_tournament_w': 0.8193976618959419}
# Best Hyperparameters: {'init_rating_m': 1288, 'k_m': 119, 'weights_regular_m': 1.1419791822297913, 'weights_tournament_m': 0.5484016402812285}



## Functions of ELO

In [19]:
def calculate_elo(teams, data, initial_rating=2000, k=140, alpha=None, weights=False, nan_score=1):
    '''
    Calculate Elo ratings for each team based on match data.

    Parameters:
    - teams (array-like): Containing Team-IDs.
    - data (pd.DataFrame): DataFrame with all matches in chronological order.
    - initial_rating (float): Initial rating of an unranked team (default: 2000).
    - k (float): K-factor, determining the impact of each match on team ratings (default: 140).
    - alpha (float or None): Tuning parameter for the multiplier for the margin of victory. No multiplier if None.

    Returns: 
    - list: Historical ratings of the winning team (WTeam).
    - list: Historical ratings of the losing team (LTeam).
    '''
    
    # Dictionary to keep track of current ratings for each team
    team_dict = {}
    for team in teams:
        team_dict[team] = initial_rating
        
    # Lists to store ratings for each team in each game
    r1, r2 = [], []
    loss = []
    margin_of_victory = 1
    weight = 1

    # Iterate through the game data
    for wteam, lteam, ws, ls, w  in tqdm(zip(data.WTeamID, data.LTeamID, data.WScore, data.LScore, data.weight), total=len(data)):

        # Calculate expected outcomes based on Elo ratings
        rateW = 1 / (1 + 10 ** ((team_dict[lteam] - team_dict[wteam]) / initial_rating))
        rateL = 1 / (1 + 10 ** ((team_dict[wteam] - team_dict[lteam]) / initial_rating))
        
        if alpha:
                margin_of_victory = (ws - ls)/alpha
        if isinstance(weights, (list, np.ndarray, pd.Series)):
            weight = w

        # Update ratings for winning and losing teams
        team_dict[wteam] += w * k * margin_of_victory * (1 - rateW)
        team_dict[lteam] += w * k * margin_of_victory * (0 - rateL)

        # Ensure that ratings do not go below 1
        if team_dict[lteam] < 1:
            team_dict[lteam] = 1
            
        # Append current ratings for teams to lists
        r1.append(team_dict[wteam])
        r2.append(team_dict[lteam])
        loss.append((1-rateW)**2)
        
    return r1, r2, loss

def create_elo_data(teams, data, initial_rating=2000, k=140, alpha=None, weights=None, nan_score=1):
    '''
    Create a DataFrame with summary statistics of Elo ratings for teams based on historical match data.

    Parameters:
    - teams (array-like): Containing Team-IDs.
    - data (pd.DataFrame): DataFrame with all matches in chronological order.
    - initial_rating (float): Initial rating of an unranked team (default: 2000).
    - k (float): K-factor, determining the impact of each match on team ratings (default: 140).
    - weights (array-like): Containing weights for each match.

    Returns: 
    - DataFrame: Summary statistics of Elo ratings for teams throughout a season.
    '''
    
    if isinstance(weights, (list, np.ndarray, pd.Series)):
        data['weight'] = weights
    else:
        data['weight'] = 1
    
    r1, r2, loss = calculate_elo(teams, data, initial_rating, k, alpha, weights, nan_score)
    # Calculate loss only on tourney results
    loss = np.mean(np.array(loss)[data.tourney == 1])
    print(f"Loss: {loss}")
    
    # Concatenate arrays vertically
    seasons = np.concatenate([data.Season, data.Season])
    days = np.concatenate([data.DayNum, data.DayNum])
    teams = np.concatenate([data.WTeamID, data.LTeamID])
    tourney = np.concatenate([data.tourney, data.tourney])
    ratings = np.concatenate([r1, r2])
    # Create a DataFrame
    rating_df = pd.DataFrame({
        'Season': seasons,
        'DayNum': days,
        'TeamID': teams,
        'Rating': ratings,
        'Tourney': tourney
    })

    # Sort DataFrame and remove tournament data
    rating_df.sort_values(['TeamID', 'Season', 'DayNum'], inplace=True)
    rating_df = rating_df[rating_df['Tourney'] == 0]
    grouped = rating_df.groupby(['TeamID', 'Season'])
    results = grouped['Rating'].agg(['mean', 'median', 'std', 'min', 'max', 'last'])
    results.columns = ['Rating_Mean', 'Rating_Median', 'Rating_Std', 'Rating_Min', 'Rating_Max', 'Rating_Last']
    results['Rating_Trend'] = grouped.apply(lambda x: linregress(range(len(x)), x['Rating']).slope, include_groups=False)
    results.reset_index(inplace=True)
    
    return results, loss

def generate_2025_match_predictions(elo_df, teams, season=2025):
    predictions = []
    
    for i in range(len(teams)):
        for j in range(i + 1, len(teams)):
            team1_id = teams[i]
            team2_id = teams[j]
            
            # 获取两支队伍的 Elo 评分
            rating_team1 = elo_df[(elo_df['TeamID'] == team1_id) & (elo_df['Season'] == season)]['Rating_Last'].values[0]
            rating_team2 = elo_df[(elo_df['TeamID'] == team2_id) & (elo_df['Season'] == season)]['Rating_Last'].values[0]
            
            # 计算胜率
            expected_score_team1 = 1 / (1 + 10 ** ((rating_team2 - rating_team1) / 400))
            
            # 生成 ID
            match_id = f"{season}_{team1_id}_{team2_id}"
            
            # 添加到预测列表
            predictions.append([match_id, expected_score_team1])
    
    # 创建 DataFrame
    predictions_df = pd.DataFrame(predictions, columns=['ID', 'Pred'])
    return predictions_df

def generate_match_predictions(elo_df, match_data):
    """
    根据 ELO 分数计算比赛的胜率。
    
    参数:
        elo_df (pd.DataFrame): 包含队伍 ELO 分数的 DataFrame，列包括 ['Season', 'TeamID', 'ELO']。
        match_data (pd.DataFrame): 包含比赛数据的 DataFrame，列包括 ['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WScore', 'LScore', 'WLoc', 'NumOT', 'tourney', 'weight']。
    
    返回:
        pd.DataFrame: 包含比赛预测结果的 DataFrame，列包括 ['Season', 'WTeamID', 'LTeamID', 'WTeamELO', 'LTeamELO', 'WinProbability']。
    """
    # 初始化结果 DataFrame
    results = []

    # 遍历每场比赛
    for _, row in match_data.iterrows():
        season = row['Season']
        w_team_id = row['WTeamID']
        l_team_id = row['LTeamID']

        # 提取对应赛季的 ELO 分数
        w_team_elo = elo_df[(elo_df['Season'] == season) & (elo_df['TeamID'] == w_team_id)]['Rating_Mean'].values
        l_team_elo = elo_df[(elo_df['Season'] == season) & (elo_df['TeamID'] == l_team_id)]['Rating_Mean'].values

        # 检查是否找到 ELO 分数
        if len(w_team_elo) == 0 or len(l_team_elo) == 0:
            print(f"ELO 分数未找到：赛季 {season}, 队伍 {w_team_id} 或 {l_team_id}")
            w_team_elo = elo_df[(elo_df['Season'] == season) & (elo_df['TeamID'] == w_team_id)]['Rating_Last'].values
            l_team_elo = elo_df[(elo_df['Season'] == season) & (elo_df['TeamID'] == l_team_id)]['Rating_Last'].values
            continue

        w_team_elo = w_team_elo[0]
        l_team_elo = l_team_elo[0]

        # 计算胜率
        # 使用 ELO 公式：P(A 胜 B) = 1 / (1 + 10^((ELO_B - ELO_A) / 400))
        win_probability = 1 / (1 + 10 ** ((l_team_elo - w_team_elo) / 400))

        # 将结果存储到列表中
        results.append({
            'Season': season,
            'WTeamID': w_team_id,
            'LTeamID': l_team_id,
            'WTeamELO': w_team_elo,
            'LTeamELO': l_team_elo,
            'WinProbability': win_probability
        })

    # 将结果列表转换为 DataFrame
    result_df = pd.DataFrame(results)
    return result_df

## EDA for ELO

In [7]:
regular_m['tourney'] = 0
tourney_m['tourney'] = 1
regular_m['weight'] = weights_regular_m
tourney_m['weight'] = weights_tournament_m

data_m = pd.concat([regular_m, tourney_m])
data_m.sort_values(['Season', 'DayNum'], inplace=True)
data_m.reset_index(inplace=True, drop=True)

regular_w['tourney'] = 0
tourney_w['tourney'] = 1
regular_w['weight'] = weights_regular_w
tourney_w['weight'] = weights_tournament_w

data_w = pd.concat([regular_w, tourney_w])
data_w.sort_values(['Season', 'DayNum'], inplace=True)
data_w.reset_index(inplace=True, drop=True)

In [8]:
print(teams_m['TeamID'].head())
print(data_m.tail())

0    1101
1    1102
2    1103
3    1104
4    1105
Name: TeamID, dtype: int64
        Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT  tourney  \
194309    2025     106     1461      69     1102      62    H      0        0   
194310    2025     106     1462      76     1139      63    H      0        0   
194311    2025     106     1466      80     1480      62    H      0        0   
194312    2025     106     1468      94     1122      68    H      0        0   
194313    2025     106     1474      89     1146      72    H      0        0   

        weight  
194309     1.0  
194310     1.0  
194311     1.0  
194312     1.0  
194313     1.0  


## Predictions of ELO

In [ ]:
elo_df_men, _ = create_elo_data(teams_m.TeamID, data_m, initial_rating=init_rating_m, k=k_m, alpha=alpha_m, weights=data_m['weight'])
elo_df_women, _ = create_elo_data(teams_w.TeamID, data_w, initial_rating=init_rating_w, k=k_w, alpha=alpha_w, weights=data_w['weight'])


# 生成男子组比赛预测
men_teams_2025 = teams_m[teams_m['TeamID'].isin(elo_df_men[elo_df_men['Season'] == 2025]['TeamID'])]['TeamID'].values
men_predictions = generate_2025_match_predictions(elo_df_men, men_teams_2025)

# 生成女子组比赛预测
women_teams_2025 = teams_w[teams_w['TeamID'].isin(elo_df_women[elo_df_women['Season'] == 2025]['TeamID'])]['TeamID'].values
women_predictions = generate_2025_match_predictions(elo_df_women, women_teams_2025)

# 合并所有预测
all_predictions = pd.concat([men_predictions, women_predictions])

pred_of_ELO = all_predictions


In [26]:
data_m

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,tourney,weight
0,1985,20,1228,81,1328,64,N,0,0,1.0
1,1985,25,1106,77,1354,70,H,0,0,1.0
2,1985,25,1112,63,1223,56,H,0,0,1.0
3,1985,25,1165,70,1432,54,H,0,0,1.0
4,1985,25,1192,86,1447,74,H,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...
194309,2025,106,1461,69,1102,62,H,0,0,1.0
194310,2025,106,1462,76,1139,63,H,0,0,1.0
194311,2025,106,1466,80,1480,62,H,0,0,1.0
194312,2025,106,1468,94,1122,68,H,0,0,1.0


## Crossval for ELO

In [30]:
train_df, test_df = train_test_split(data_m, test_size=0.01, random_state=874, stratify=data_m['Season'])
# train_df, test_df = train_test_split(data_m, test_size=0.05, random_state=42, stratify=None, shuffle=False)
train_df = train_df.sort_values(by='Season')
# 3. 构建 ELO 模型
# 使用训练集构建 ELO 分数
# 假设所有队伍的初始 ELO 分数为初始值，例如 1500
initial_rating = 1500
k = 120  # 调整参数
alpha = 10.1  # 调整参数

# 提取所有队伍的 ID
teams = pd.concat([train_df['WTeamID'], train_df['LTeamID']]).unique()

# 创建 ELO 数据
elo_df, _ = create_elo_data(teams, train_df, initial_rating=initial_rating, k=k, alpha=alpha, weights=train_df['weight'])

# 4. 生成测试集的预测
# 提取测试集中所有队伍
test_teams = np.array(sorted(pd.concat([test_df['WTeamID'], test_df['LTeamID']]).unique()))




100%|██████████| 192370/192370 [00:00<00:00, 306937.26it/s]


Loss: 0.20064670800352777


In [18]:
elo_df

,TeamID,Season,Rating_Mean,Rating_Median,Rating_Std,Rating_Min,Rating_Max,Rating_Last,Rating_Trend
0,1101,2014,1022.434646,1002.126285,194.726983,714.662853,1415.523182,974.619528,-2.360962
1,1101,2015,526.171849,528.438036,48.781895,418.764534,647.376160,515.136008,-0.367596
2,1101,2016,570.881261,555.765886,53.831340,493.812669,668.459153,493.812669,-3.040630
3,1101,2017,684.097064,693.883524,75.867878,506.683465,846.521323,506.683465,-3.289195
4,1101,2018,843.098899,886.531004,146.237754,537.012638,1065.404025,1020.452028,1.074494
...,...,...,...,...,...,...,...,...,...
13383,1477,2025,373.115391,377.846111,104.162978,180.576366,562.818030,180.576366,0.403862
13384,1478,2024,1128.968154,1100.181782,120.065138,928.160772,1392.062984,1181.161603,6.344942
13385,1478,2025,813.814606,829.432813,99.887290,593.495363,983.686372,874.358394,-2.921788
13386,1479,2025,1152.973653,1139.798423,132.520391,937.908480,1478.093875,1377.042722,4.563133


In [31]:
# 生成预测
predictions = generate_match_predictions(elo_df, test_df)

# 5. 评估预测性能
# 创建一个新列，表示比赛结果（1 表示 WTeamID 获胜，0 表示 LTeamID 获胜）
test_df['result'] = 1

# 计算预测准确率
# 假设 predictions 是一个 DataFrame，包含预测结果
# 需要将预测结果与实际结果进行对比
# 例如，predictions 应该包含 'WTeamID', 'LTeamID', 'predicted_winner' 等列

# 将测试集的实际结果与预测结果合并
#test_df = test_df.merge(predictions, on=['WTeamID', 'LTeamID'], how='left')

# 计算准确率

import random
# predictions['WinProbability'] = np.random.rand(len(predictions))

brierscore_elo = brier_score_loss(test_df['result'], predictions['WinProbability'])
print(f"brier score: {brierscore_elo:.2f}")



brier score: 0.24


In [28]:
import random
predictions['WinProbability'] = np.random.rand(len(predictions))
predictions['WinProbability']

0       0.414288
1       0.955987
2       0.705376
3       0.607257
4       0.770818
          ...   
9711    0.208172
9712    0.636051
9713    0.582616
9714    0.105872
9715    0.246067
Name: WinProbability, Length: 9716, dtype: float64

## Optuna for ELO

In [33]:
def objective_elo_m(trial):
    # 定义需要优化的参
    params_elo_m = {
        # 男性相关参数
        'init_rating_m': trial.suggest_int('init_rating_m', 1000, 1500),  # 初始评分（男性）
        'k_m': trial.suggest_int('k_m', 100, 250),  # K 值（男性）
        'weights_regular_m': trial.suggest_float('weights_regular_m', 0.5, 1.5),  # 普通比赛权重（男性）
        'weights_tournament_m': trial.suggest_float('weights_tournament_m', 0.5, 1.5),  # 锦标赛权重（男性）
        'alpha_m': None,  # 男性相关固定参数

    }

    init_rating_m, k_m, weights_regular_m, weights_tournament_m, alpha_m =params_elo_m.values()

    regular_m['tourney'] = 0
    tourney_m['tourney'] = 1
    regular_m['weight'] = weights_regular_m
    tourney_m['weight'] = weights_tournament_m

    data_m = pd.concat([regular_m, tourney_m])
    data_m.sort_values(['Season', 'DayNum'], inplace=True)
    data_m.reset_index(inplace=True, drop=True)

    _, loss = create_elo_data(teams_m.TeamID, data_m, initial_rating=init_rating_m, k=k_m, alpha=alpha_m, weights=data_m['weight'])

    return loss

# 创建 Optuna 研究对象
study_elo_m = optuna.create_study(direction='minimize')  # 最小化 MSE

# 运行优化
study_elo_m.optimize(objective_elo_m, n_trials=50, show_progress_bar=True)  # 运行 50 次试验

# 获取最佳超参数
best_params_elo_m = study_elo_m.best_params
best_score_elo_m = study_elo_m.best_value

print("Best Hyperparameters:", best_params_elo_m)
print("Best MSE:", best_score_elo_m)

def objective_elo_w(trial):
    # 定义需要优化的参
    params_elo_w = {
        # 女性相关参数
        'init_rating_w': trial.suggest_int('init_rating_w', 1000, 1500),  # 初始评分（女性）
        'k_w': trial.suggest_int('k_w', 100, 250),  # K 值（女性）
        'weights_regular_w': trial.suggest_float('weights_regular_w', 0.5, 1.5),  # 普通比赛权重（女性）
        'weights_tournament_w': trial.suggest_float('weights_tournament_w', 0.5, 1.5),  # 锦标赛权重（女性）
        'alpha_w': None,  # 女性相关固定参数
    }

    init_rating_w, k_w, weights_regular_w, weights_tournament_w, alpha_w =params_elo_w.values()

    regular_w['tourney'] = 0
    tourney_w['tourney'] = 1
    regular_w['weight'] = weights_regular_w
    tourney_w['weight'] = weights_tournament_w

    data_w = pd.concat([regular_w, tourney_w])
    data_w.sort_values(['Season', 'DayNum'], inplace=True)
    data_w.reset_index(inplace=True, drop=True)

    _, loss = create_elo_data(teams_w.TeamID, data_w, initial_rating=init_rating_w, k=k_w, alpha=alpha_w, weights=data_w['weight'])

    return loss

# 创建 Optuna 研究对象
study_elo_w = optuna.create_study(direction='minimize')  # 最小化 MSE

# 运行优化
study_elo_w.optimize(objective_elo_w, n_trials=50, show_progress_bar=True)  # 运行 50 次试验

# 获取最佳超参数
best_params_elo_w = study_elo_w.best_params
best_score_elo_w = study_elo_w.best_value

print("Best Hyperparameters:", best_params_elo_w)
print("Best MSE:", best_score_elo_w)


[I 2025-03-11 18:28:22,408] A new study created in memory with name: no-name-8e00e49b-208a-4307-913d-61fdcc35f28b


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 194314/194314 [00:00<00:00, 652871.90it/s]


Loss: 0.18856764993309508
[I 2025-03-11 18:28:24,587] Trial 0 finished with value: 0.18856764993309508 and parameters: {'init_rating_m': 1339, 'k_m': 129, 'weights_regular_m': 0.9667691858259447, 'weights_tournament_m': 1.1877917159721196}. Best is trial 0 with value: 0.18856764993309508.


100%|██████████| 194314/194314 [00:00<00:00, 630746.37it/s]


...

Loss: 0.14845594485093389
[I 2025-03-11 18:31:30,759] Trial 47 finished with value: 0.14845594485093389 and parameters: {'init_rating_w': 1330, 'k_w': 125, 'weights_regular_w': 1.3143793560643948, 'weights_tournament_w': 0.9113537714329933}. Best is trial 34 with value: 0.14823002216658998.


100%|██████████| 137598/137598 [00:00<00:00, 622118.86it/s]


Loss: 0.14839890783433998
[I 2025-03-11 18:31:32,333] Trial 48 finished with value: 0.14839890783433998 and parameters: {'init_rating_w': 1310, 'k_w': 118, 'weights_regular_w': 1.3985419459880926, 'weights_tournament_w': 0.7428162264758171}. Best is trial 34 with value: 0.14823002216658998.


100%|██████████| 137598/137598 [00:00<00:00, 636494.30it/s]


Loss: 0.14845288814426785
[I 2025-03-11 18:31:33,909] Trial 49 finished with value: 0.14845288814426785 and parameters: {'init_rating_w': 1197, 'k_w': 171, 'weights_regular_w': 0.9142697596420633, 'weights_tournament_w': 1.0014575736465894}. Best is trial 34 with value: 0.14823002216658998.
Best Hyperparameters: {'init_rating_w': 1283, 'k_w': 129, 'weights_regular_w': 1.4277066442472455, 'weights_tournament_w': 0.8193976618959419}
Best MSE: 0.14823002216658998


# RandomForest

In [26]:
X = games[col].fillna(-1)
sub_X = sub[col].fillna(-1)
y = games['Pred']

param_grid_rf = {
    'n_estimators': 100,
    'max_depth': None
}



pipeline_rf = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        #('catboost', CatBoostClassifier(**param_grid_cat, verbose=False, task_type='GPU', loss_function='Logloss'))
        ('catboost', RandomForestRegressor(**param_grid_rf, verbose=False))
    ])

n_splits = 5  # 5折交叉验证
tscv = TimeSeriesSplit(n_splits=n_splits)
brier = []
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Fold {fold + 1}/{n_splits}")
    
    # 按索引拆分训练集和测试集
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = games['Pred'].iloc[train_index], games['Pred'].iloc[test_index]
    #print(len(X_train), len(X_test), len(y_train), len(y_test))

    pipeline_rf.fit(X_train, y_train)

    pred_rf = pipeline_rf.predict(X_test).clip(0.001, 0.999)


    brier_score_rf = brier_score_loss(y_test, pred_rf)



print(brier_score_rf)

Fold 1/5
Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5
0.16321057255936677


## optuna rf

In [27]:
X = games[col].fillna(-1)
sub_X = sub[col].fillna(-1)
y = games['Pred']


def objective_rf(trial):
    # 定义需要优化的参数
    param_grid_rf = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),  # 树的数量
        'max_depth': trial.suggest_int('max_depth', 2, 20),  # 树的最大深度
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),  # 分裂内部节点所需的最小样本数
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)  # 在叶子节点处需要的最小样本数
    }

    # 定义 Pipeline
    pipeline_rf = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('rf', RandomForestRegressor(**param_grid_rf, random_state=42))
    ])

    n_splits = 5  # 5折交叉验证
    tscv = TimeSeriesSplit(n_splits=n_splits)
    brier_scores = []
    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        # 按索引拆分训练集和测试集
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        pipeline_rf.fit(X_train, y_train)
        pred_rf = pipeline_rf.predict(X_test).clip(0.001, 0.999)
        brier_score_rf = brier_score_loss(y_test, pred_rf)
        brier_scores.append(brier_score_rf)

    return sum(brier_scores[-2:]) / 2  # 返回最后两次的平均 Brier 分数

# 创建 Optuna 研究对象
study_rf = optuna.create_study(direction='minimize')  # 最小化 Brier 分数

# 运行优化
study_rf.optimize(objective_rf, n_trials=50, show_progress_bar=True)  # 运行 50 次试验

# 获取最佳超参数
best_params_rf = study_rf.best_params
best_score_rf = study_rf.best_value

print("Best Hyperparameters:", best_params_rf)
print("Best Brier Score:", best_score_rf)

[I 2025-03-17 10:38:22,856] A new study created in memory with name: no-name-0928a2cc-d071-4cd5-9970-46ec62041e0f


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-03-17 10:41:52,318] Trial 0 finished with value: 0.15700734464138366 and parameters: {'n_estimators': 937, 'max_depth': 13, 'min_samples_split': 10, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.15700734464138366.
[I 2025-03-17 10:42:16,614] Trial 1 finished with value: 0.15230706230697333 and parameters: {'n_estimators': 157, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.15230706230697333.
[I 2025-03-17 10:44:24,996] Trial 2 finished with value: 0.1526384154573162 and parameters: {'n_estimators': 973, 'max_depth': 6, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.15230706230697333.
[I 2025-03-17 10:46:46,615] Trial 3 finished with value: 0.15477946613084537 and parameters: {'n_estimators': 818, 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.15230706230697333.
[I 2025-03-17 10:49:32,214] Trial 4 finished with value: 0.15619098602541276 and para

In [ ]:
trials = study_rf.trials

# 按照目标值（MSE）从小到大排序
sorted_trials = sorted(trials, key=lambda t: t.value)

# 选择前 5 次最佳试验
best_trials = sorted_trials[:5]

# 提取这 5 次试验的参数
best_params_list = [trial.params for trial in best_trials]
average_params = {}
for param in best_params_list[0].keys():  # 假设所有试验的参数名称一致
    values = [trial[param] for trial in best_params_list]
    # import ipdb; ipdb.set_trace()
    if all(isinstance(value, (int, float)) for value in values):
        average_params[param] = np.mean(values)
    else:
        # 如果参数不是数值类型，可以选择跳过或单独处理
        average_params[param] = None  # 或者使用其他方式处理

print("Average of Best 5 Hyperparameters:", average_params)

# Average of Best 5 Hyperparameters: {'n_estimators': 584.2, 'max_depth': 6.2, 'min_samples_split': 7.6, 'min_samples_leaf': 9.8}

Average of Best 5 Hyperparameters: {'n_estimators': 584.2, 'max_depth': 6.2, 'min_samples_split': 7.6, 'min_samples_leaf': 9.8}


In [29]:
st = study_rf

optuna.visualization.plot_optimization_history(st).show()
optuna.visualization.plot_param_importances(st).show()
optuna.visualization.plot_contour(st).show()
# 绘制平行坐标图
optuna.visualization.plot_parallel_coordinate(st).show()

optuna.visualization.plot_slice(st).show()

# LightGBM

In [ ]:
X = games[col].fillna(-1)
sub_X = sub[col].fillna(-1)
y = games['Pred']

param_grid_lgb = {
    'num_leaves': 31,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'max_depth': -1,
    'min_child_samples': 20
}



pipeline_lgb = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        #('catboost', CatBoostClassifier(**param_grid_cat, verbose=False, task_type='GPU', loss_function='Logloss'))
        ('catboost', LGBMRegressor(**param_grid_lgb))
    ])

n_splits = 5  # 5折交叉验证
tscv = TimeSeriesSplit(n_splits=n_splits)
brier = []
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Fold {fold + 1}/{n_splits}")
    
    # 按索引拆分训练集和测试集
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = games['Pred'].iloc[train_index], games['Pred'].iloc[test_index]
    #print(len(X_train), len(X_test), len(y_train), len(y_test))

    pipeline_lgb.fit(X_train, y_train)

    pred_lgb = pipeline_lgb.predict(X_test).clip(0.001, 0.999)


    brier_score_lgb = brier_score_loss(y_test, pred_lgb)
    print(brier_score_lgb)




print(brier_score_lgb)

Fold 1/5
0.23551049900811025
Fold 2/5
0.23149152972638376
Fold 3/5
0.22163895390041607
Fold 4/5
0.17116477400650412
Fold 5/5
0.1676247289201588
0.1676247289201588


## optuna lgb

In [ ]:
# 假设 X 和 y 已经定义好
X = games[col].fillna(-1)
y = games['Pred']

def objective_lgb(trial):
    # 定义需要优化的参数
    param_grid_lgb = {
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),  # 叶子节点的数量
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),  # 学习率
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),  # 迭代次数
        'max_depth': trial.suggest_int('max_depth', -1, 15),  # 树的最大深度
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),  # 叶子节点所需的最小样本数
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # 子采样率
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)  # 特征采样率
    }

    # 定义 Pipeline
    pipeline_lgb = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('lgb', LGBMRegressor(**param_grid_lgb, random_state=42, device='gpu'))
    ])

    n_splits = 5  # 5折交叉验证
    tscv = TimeSeriesSplit(n_splits=n_splits)
    brier_scores = []
    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        # 按索引拆分训练集和测试集
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        pipeline_lgb.fit(X_train, y_train)
        pred_lgb = pipeline_lgb.predict(X_test).clip(0.001, 0.999)
        brier_score_lgb = brier_score_loss(y_test, pred_lgb)
        brier_scores.append(brier_score_lgb)

    return sum(brier_scores[-2:]) / 2  # 返回最后两次的平均 Brier 分数

# 创建 Optuna 研究对象
study_lgb = optuna.create_study(direction='minimize')  # 最小化 Brier 分数

# 运行优化
study_lgb.optimize(objective_lgb, n_trials=50, show_progress_bar=True)  # 运行 50 次试验

# 获取最佳超参数
best_params_lgb = study_lgb.best_params
best_score_lgb = study_lgb.best_value

print("Best Hyperparameters:", best_params_lgb)
print("Best Brier Score:", best_score_lgb)

# Best Hyperparameters: {'num_leaves': 84, 'learning_rate': 0.0057642951478954745, 'n_estimators': 637, 'max_depth': 0, 'min_child_samples': 79, 'subsample': 0.9791450095293126, 'colsample_bytree': 0.8859090843459869}
# Best Brier Score: 0.15150768537962822

[I 2025-03-17 11:58:28,432] A new study created in memory with name: no-name-553c8877-2cac-4a60-be70-85e17cf990d3


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-03-17 11:58:33,997] Trial 0 finished with value: 0.17399501540369444 and parameters: {'num_leaves': 80, 'learning_rate': 0.0028941884313159584, 'n_estimators': 676, 'max_depth': 1, 'min_child_samples': 67, 'subsample': 0.9008617461189479, 'colsample_bytree': 0.7679455365517477}. Best is trial 0 with value: 0.17399501540369444.
[I 2025-03-17 11:58:35,753] Trial 1 finished with value: 0.15771330919873788 and parameters: {'num_leaves': 70, 'learning_rate': 0.027087463497899533, 'n_estimators': 270, 'max_depth': 2, 'min_child_samples': 14, 'subsample': 0.8989962171817288, 'colsample_bytree': 0.8863308433036643}. Best is trial 1 with value: 0.15771330919873788.
[I 2025-03-17 11:58:42,977] Trial 2 finished with value: 0.17827748675008984 and parameters: {'num_leaves': 44, 'learning_rate': 0.06261772110069136, 'n_estimators': 839, 'max_depth': 5, 'min_child_samples': 42, 'subsample': 0.5547646598020954, 'colsample_bytree': 0.6273596114476534}. Best is trial 1 with value: 0.15771330919

In [ ]:
trials = study_lgb.trials

# 按照目标值（MSE）从小到大排序
sorted_trials = sorted(trials, key=lambda t: t.value)

# 选择前 5 次最佳试验
best_trials = sorted_trials[:5]

# 提取这 5 次试验的参数
best_params_list = [trial.params for trial in best_trials]
average_params = {}
for param in best_params_list[0].keys():  # 假设所有试验的参数名称一致
    values = [trial[param] for trial in best_params_list]
    # import ipdb; ipdb.set_trace()
    if all(isinstance(value, (int, float)) for value in values):
        average_params[param] = np.mean(values)
    else:
        # 如果参数不是数值类型，可以选择跳过或单独处理
        average_params[param] = None  # 或者使用其他方式处理

print("Average of Best 5 Hyperparameters:", average_params)


# Average of Best 5 Hyperparameters: {'num_leaves': 63.2, 'learning_rate': 0.007602565555810267, 'n_estimators': 537.6, 'max_depth': 2.8, 'min_child_samples': 82.0, 'subsample': 0.8191854478457833, 'colsample_bytree': 0.8521050862477741}


Average of Best 5 Hyperparameters: {'num_leaves': 63.2, 'learning_rate': 0.007602565555810267, 'n_estimators': 537.6, 'max_depth': 2.8, 'min_child_samples': 82.0, 'subsample': 0.8191854478457833, 'colsample_bytree': 0.8521050862477741}


In [32]:
st = study_lgb

optuna.visualization.plot_optimization_history(st).show()
optuna.visualization.plot_param_importances(st).show()
optuna.visualization.plot_contour(st).show()
# 绘制平行坐标图
optuna.visualization.plot_parallel_coordinate(st).show()

optuna.visualization.plot_slice(st).show()

# Model Merge

In [15]:
# Best Hyperparameters: {'n_estimators': 346, 'learning_rate': 0.010193615589609411, 'max_depth': 3}
X = games[col].fillna(-1)
sub_X = sub[col].fillna(-1)
y = games['Pred']
#X = all_results[['Season', 'WTeamID', 'LTeamID']] # for cat?


In [11]:
X

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325
...,...,...,...
8331,2024,3425,3163
8332,2024,3261,3234
8333,2024,3163,3234
8334,2024,3301,3376


In [35]:


# 定义需要优化的参数
param_grid_xgb = {
    'n_estimators': 346,
    'learning_rate': 0.01,
    'max_depth': 3
}

param_grid_mlp = {
    'alpha': 0.0001,
    'learning_rate_init': 0.001,
    'max_iter': 200,
    'hidden_layer_sizes': (1000, 1000)
}

param_grid_cat = {
        'iterations': 322,
        'depth': 3,
        'learning_rate': 0.025
    }

param_grid_rf = {
        'n_estimators': 584,
        'max_depth': 6,
        'min_samples_split': 8,
        'min_samples_leaf': 10
    }

param_grid_lgb = {
        'num_leaves': 63,
        'learning_rate': 0.007602565555810267,
        'n_estimators': 537,
        'max_depth': 3,
        'min_child_samples': 82,
        'subsample': 0.8191854478457833,
        'colsample_bytree': 0.8521050862477741
    }


# 定义 Pipeline
pipeline_xgb = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor(**param_grid_xgb, device="gpu", random_state=42))
])

pipeline_mlp = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # 均值填充
    ('scaler', StandardScaler()),  # 标准化
    ('model', MLPRegressor(**param_grid_mlp))  # 神经网络回归器
])

pipeline_cat = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        #('catboost', CatBoostClassifier(**param_grid_cat, verbose=False, task_type='GPU', loss_function='Logloss'))
        ('catboost', CatBoostRegressor(**param_grid_cat, verbose=False, task_type='GPU'))
    ])

pipeline_rf = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('rf', RandomForestRegressor(**param_grid_rf, random_state=42))
    ])

pipeline_lgb = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('lgb', LGBMRegressor(**param_grid_lgb, random_state=42, device='gpu'))
    ])



n_splits = 5  # 5折交叉验证
tscv = TimeSeriesSplit(n_splits=n_splits)
brier = []
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Fold {fold + 1}/{n_splits}")
    
    # 按索引拆分训练集和测试集
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = games['Pred'].iloc[train_index], games['Pred'].iloc[test_index]
    #print(len(X_train), len(X_test), len(y_train), len(y_test))
    pipeline_xgb.fit(X_train, y_train)
    pipeline_mlp.fit(X_train, y_train)
    pipeline_cat.fit(X_train, y_train)
    pipeline_rf.fit(X_train, y_train)
    pipeline_lgb.fit(X_train, y_train)


    pred_xgb = pipeline_xgb.predict(X_test).clip(0.001, 0.999)
    pred_mlp = pipeline_mlp.predict(X_test).clip(0.001, 0.999)
    pred_cat = pipeline_cat.predict(X_test).clip(0.001, 0.999)
    pred_rf = pipeline_rf.predict(X_test).clip(0.001, 0.999)
    pred_lgb = pipeline_lgb.predict(X_test).clip(0.001, 0.999)


    brier_score_xgb = brier_score_loss(y_test, pred_xgb)
    brier_score_mlp = brier_score_loss(y_test, pred_mlp)
    brier_score_cat = brier_score_loss(y_test, pred_cat)
    brier_score_rf = brier_score_loss(y_test, pred_rf)
    brier_score_lgb = brier_score_loss(y_test, pred_lgb)



    mean = (pred_xgb + pred_mlp + pred_cat + pred_rf + pred_lgb) / 5.0
    brier_score_mean = brier_score_loss(y_test, mean)

print(f'xgb: {brier_score_xgb:.3f}, mlp: {brier_score_mlp:.3f}, cat: {brier_score_cat:.3f}, rf: {brier_score_rf:.3f}, lgb: {brier_score_lgb:.3f}, mean: {brier_score_mean:.3f}')




Fold 1/5
Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5
xgb: 0.154, mlp: 0.243, cat: 0.153, rf: 0.151, lgb: 0.153, mean: 0.160


## Finding weights

In [ ]:
X = games[col].fillna(-1)
sub_X = sub[col].fillna(-1)
y = games['Pred']
# X = all_results[['Season', 'WTeamID', 'LTeamID']] # for cat?


# 定义需要优化的参数
param_grid_xgb = {
    'n_estimators': 346,
    'learning_rate': 0.01,
    'max_depth': 3
}

param_grid_mlp = {
    'alpha': 0.0001,
    'learning_rate_init': 0.001,
    'max_iter': 200,
    'hidden_layer_sizes': (1000, 1000)
}

param_grid_cat = {
        'iterations': 322,
        'depth': 3,
        'learning_rate': 0.025
    }

param_grid_rf = {
        'n_estimators': 584,
        'max_depth': 6,
        'min_samples_split': 8,
        'min_samples_leaf': 10
    }

param_grid_lgb = {
        'num_leaves': 63,
        'learning_rate': 0.007602565555810267,
        'n_estimators': 537,
        'max_depth': 3,
        'min_child_samples': 82,
        'subsample': 0.8191854478457833,
        'colsample_bytree': 0.8521050862477741
    }


# 定义 Pipeline
pipeline_xgb = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor(**param_grid_xgb, device="gpu", random_state=42))
])

pipeline_mlp = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # 均值填充
    ('scaler', StandardScaler()),  # 标准化
    ('model', MLPRegressor(**param_grid_mlp))  # 神经网络回归器
])

pipeline_cat = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        #('catboost', CatBoostClassifier(**param_grid_cat, verbose=False, task_type='GPU', loss_function='Logloss'))
        ('catboost', CatBoostRegressor(**param_grid_cat, verbose=False, task_type='GPU'))
    ])

pipeline_rf = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('rf', RandomForestRegressor(**param_grid_rf, random_state=42))
    ])

pipeline_lgb = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('lgb', LGBMRegressor(**param_grid_lgb, random_state=42, device='gpu'))
    ])


split_index = int(len(X) * 0.97)  # 假设 e 占总数据的 20%，可以根据实际情况调整

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

pipeline_xgb.fit(X_train, y_train)
pipeline_mlp.fit(X_train, y_train)
pipeline_cat.fit(X_train, y_train)
pipeline_rf.fit(X_train, y_train)
pipeline_lgb.fit(X_train, y_train)


pred_xgb = pipeline_xgb.predict(X_test).clip(0.001, 0.999)
pred_mlp = pipeline_mlp.predict(X_test).clip(0.001, 0.999)
pred_cat = pipeline_cat.predict(X_test).clip(0.001, 0.999)
pred_rf = pipeline_rf.predict(X_test).clip(0.001, 0.999)
pred_lgb = pipeline_lgb.predict(X_test).clip(0.001, 0.999)


brier_score_xgb = brier_score_loss(y_test, pred_xgb)
brier_score_mlp = brier_score_loss(y_test, pred_mlp)
brier_score_cat = brier_score_loss(y_test, pred_cat)
brier_score_rf = brier_score_loss(y_test, pred_rf)
brier_score_lgb = brier_score_loss(y_test, pred_lgb)




# 打印结果
# print(f"xgb: {brier_score_xgb:.3f}, mlp: {brier_score_mlp:.3f}, cat: {brier_score_cat:.3f}, mean: {brier_score_mean:.3f}")


[I 2025-03-17 15:35:43,798] A new study created in memory with name: no-name-08eb5694-387d-4155-84ce-4b9ee846993d


In [47]:
study = optuna.create_study(direction='minimize')
def obj_weights(trial):
    # 定义权重参数
    xgb_weight = trial.suggest_float('xgb_weight', 0, 1)
    mlp_weight = trial.suggest_float('mlp_weight', 0, 1 - xgb_weight)
    cat_weight = trial.suggest_float('cat_weight', 0, 1 - xgb_weight - mlp_weight)
    rf_weight = trial.suggest_float('rf_weight', 0, 1 - xgb_weight - mlp_weight - cat_weight)
    lgb_weight = 1 - xgb_weight - mlp_weight - cat_weight - rf_weight

    # 计算加权平均预测
    mean_pred = (
        xgb_weight * pred_xgb +
        mlp_weight * pred_mlp +
        cat_weight * pred_cat +
        rf_weight * pred_rf +
        lgb_weight * pred_lgb
    )

    # 计算 Brier Score Loss
    brier_score_mean = brier_score_loss(y_test, mean_pred)
    return brier_score_mean

study.optimize(obj_weights, n_trials=50, show_progress_bar=True)

best_params = study.best_params
best_brier_score = study.best_value

print(f"Best Brier Score: {best_brier_score:.4f}")
print(f"Best Weights: {best_params}, 'cat_weight': {1 - best_params['xgb_weight'] - best_params['mlp_weight']}")

[I 2025-03-17 15:40:07,949] A new study created in memory with name: no-name-b8489a25-06bc-4adf-9028-eee2ac3052a1


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-03-17 15:40:59,644] Trial 0 finished with value: 0.17276344388273843 and parameters: {'xgb_weight': 0.2004942860296003, 'mlp_weight': 0.5973171628473434, 'cat_weight': 0.14664768533825204, 'rf_weight': 0.03843478726697979}. Best is trial 0 with value: 0.17276344388273843.
[I 2025-03-17 15:41:12,237] Trial 1 finished with value: 0.16221005400929084 and parameters: {'xgb_weight': 0.5006851067696051, 'mlp_weight': 0.40185138072023197, 'cat_weight': 0.05195445544969831, 'rf_weight': 0.021723142425228503}. Best is trial 1 with value: 0.16221005400929084.
[I 2025-03-17 15:41:14,075] Trial 2 finished with value: 0.14790645391317575 and parameters: {'xgb_weight': 0.7167452890580751, 'mlp_weight': 0.034094722337586085, 'cat_weight': 0.23007575791660004, 'rf_weight': 0.011876873997886693}. Best is trial 2 with value: 0.14790645391317575.
[I 2025-03-17 15:41:15,652] Trial 3 finished with value: 0.1559982780667775 and parameters: {'xgb_weight': 0.7181391679148014, 'mlp_weight': 0.250481519

c:\Users\Li Nuo\.conda\envs\mmmcatboost\lib\site-packages\optuna\samplers\_tpe\sampler.py:513: RuntimeWarning:

invalid value encountered in subtract



[I 2025-03-17 15:41:24,610] Trial 10 finished with value: 0.14916289240068661 and parameters: {'xgb_weight': 0.9896694850694168, 'mlp_weight': 0.006539927029453829, 'cat_weight': 0.0037688529128686588, 'rf_weight': 0.014050372823961342}. Best is trial 2 with value: 0.14790645391317575.
[I 2025-03-17 15:41:25,428] Trial 11 finished with value: 0.14912448235582007 and parameters: {'xgb_weight': 0.9748599988245671, 'mlp_weight': 0.011830606875251578, 'cat_weight': 0.013167887663925237, 'rf_weight': 1.2800566733331589e-05}. Best is trial 2 with value: 0.14790645391317575.
[I 2025-03-17 15:41:28,467] Trial 12 finished with value: 0.14890142713344579 and parameters: {'xgb_weight': 0.673154373445377, 'mlp_weight': 0.06890813676467593, 'cat_weight': 0.18705848402245104, 'rf_weight': 0.040519796374956465}. Best is trial 2 with value: 0.14790645391317575.
[I 2025-03-17 15:41:29,464] Trial 13 finished with value: 0.14923147653543323 and parameters: {'xgb_weight': 0.6452826682499927, 'mlp_weight':

c:\Users\Li Nuo\.conda\envs\mmmcatboost\lib\site-packages\optuna\samplers\_tpe\sampler.py:513: RuntimeWarning:

invalid value encountered in subtract



[W 2025-03-17 15:41:44,620] Trial 16 failed with parameters: {'xgb_weight': 0.03154502202404935, 'mlp_weight': 0.9434398472050425, 'cat_weight': 0.0249825289102725, 'rf_weight': 0.1246517780374051} because of the following error: ValueError('y_proba contains values less than 0.').
Traceback (most recent call last):
  File "c:\Users\Li Nuo\.conda\envs\mmmcatboost\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Li Nuo\AppData\Local\Temp\ipykernel_6576\212066075.py", line 20, in obj_weights
    brier_score_mean = brier_score_loss(y_test, mean_pred)
  File "c:\Users\Li Nuo\.conda\envs\mmmcatboost\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Li Nuo\.conda\envs\mmmcatboost\lib\site-packages\sklearn\metrics\_classification.py", line 3264, in brier_score_loss
    raise ValueError("y_proba contains values less than 0.")
ValueError: y_proba con

ValueError: y_proba contains values less than 0.